In [27]:
#install.packages("raster")
install.packages("terra")

Warning message in install.packages("terra"):
“installation of package ‘terra’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [28]:
#library("raster")
library("terra")

In [29]:
#country polygons
#world <- readOGR("/projects/shared-buckets/leitoldv/NaturalEarth/ne_10m_admin_0_countries.shp")
world <- terra::vect("/projects/shared-buckets/leitoldv/NaturalEarth/ne_10m_admin_0_countries.shp")

#1degree tiles
#G1deg_tiles <- readOGR("/projects/shared-buckets/leitoldv/AOIs/GRID1deg_poly_52N52S.shp")
G1deg_tiles <- terra::vect("/projects/shared-buckets/leitoldv/AOIs/GRID1deg_poly_52N52S.shp")

In [30]:
unique(world$CONTINENT)

#sort(world[world$CONTINENT=='Africa',]$ADM0_A3)
#sort(world[world$CONTINENT=='Asia',]$ADM0_A3)
#sort(world[world$CONTINENT=='Europe',]$ADM0_A3)
#sort(world[world$CONTINENT=='North America',]$ADM0_A3)
sort(world[world$CONTINENT=='South America',]$ADM0_A3)
#sort(world[world$CONTINENT=='Oceania',]$ADM0_A3)


[1] "Asia"                    "South America"          
[3] "Africa"                  "Europe"                 
[5] "North America"           "Oceania"                
[7] "Antarctica"              "Seven seas (open ocean)"

[1] "ARG" "BOL" "BRA" "CHL" "COL" "ECU" "FLK" "GUY" "PER" "PRY" "SUR" "URY"
[13] "VEN"

In [6]:
#select AOI (i.e. a country)
iso3 <- c("ARG")
country <- world[world$ADM0_A3 %in% iso3,]
aoi <- country

In [7]:
if (!identical(crs(aoi), crs(G1deg_tiles))){aoi <- project(aoi, crs(G1deg_tiles))}
intersecting <- intersect(G1deg_tiles, aoi)
  if (nrow(intersecting) == 0) {stop('no intersecting G1deg_tiles found')
  } else {
    tileindex <- intersecting$id
  }
tileindex

[1] 19219 19220 19221 19393 19394 19395 19398 19399 19400 19401 19402 19567
 [13] 19568 19569 19570 19571 19572 19573 19574 19575 19576 19577 19578 19579
 [25] 19580 19581 19582 19740 19741 19742 19743 19744 19745 19746 19747 19748
 [37] 19749 19750 19751 19752 19753 19754 19755 19756 19757 19758 19759 19760
 [49] 19761 19762 19918 19919 19920 19921 19922 19923 19924 19925 19926 19927
 [61] 19928 19929 19930 19931 19932 19933 19934 19935 19936 19937 19938 19939
 [73] 19940 19941 19942 20095 20096 20097 20098 20099 20100 20101 20102 20103
 [85] 20104 20105 20106 20107 20108 20109 20110 20111 20112 20113 20114 20115
 [97] 20116 20117 20118 20119 20120 20121 20122 20273 20274 20275 20276 20277
[109] 20278 20279 20280 20281 20282 20283 20284 20285 20286 20287 20288 20289
[121] 20290 20291 20292 20293 20294 20295 20296 20297 20298 20299 20300 20301
[133] 20452 20453 20454 20455 20456 20457 20458 20459 20460 20461 20462 20463
[145] 20464 20465 20466 20467 20468 20469 20470 20471 20472 20473 20474 20475
[157] 20476 20477 20478 20479 20632 20633 20634 20635 20636 20637 20638 20639
[169] 20640 20641 20642 20643 20644 20645 20646 20647 20648 20649 20650 20651
[181] 20652 20653 20654 20655 20656 20658 20659 20813 20814 20815 20816 20817
[193] 20818 20819 20820 20821 20822 20823 20824 20825 20826 20827 20828 20829
[205] 20830 20831 20832 20833 20834 20992 20993 20994 20995 20996 20997 20998
[217] 20999 21000 21001 21002 21003 21004 21005 21006 21007 21008 21009 21010
[229] 21011 21012 21013 21172 21173 21174 21175 21176 21177 21178 21179 21180
[241] 21181 21182 21183 21184 21185 21186 21187 21188 21189 21190 21191 21192
[253] 21353 21354 21355 21356 21357 21358 21359 21360 21361 21362 21363 21364
[265] 21365 21366 21367 21368 21369 21370 21534 21535 21536 21537 21538 21539
[277] 21540 21541 21542 21543 21544 21545 21546 21547 21548 21549 21715 21716
[289] 21717 21718 21719 21720 21721 21722 21723 21724 21725 21726 21727 21728
[301] 21729 21895 21896 21897 21898 21899 21900 21901 21902 21903 21904 21905
[313] 21906 21907 21908 21909 22076 22077 22078 22079 22080 22081 22082 22085
[325] 22086 22087 22088 22089 22258 22259 22260 22267 22268 22437 22438 22439
[337] 22616 22617 22618 22796 22797 22798

In [8]:
tilenames <- rep("NA",length(tileindex))
s3path <- rep("NA",length(tileindex))
l4path <- rep("NA",length(tileindex))

tileindex_df <- data.frame(tileindex, tilenames, s3path, l4path)
for(i in 1:nrow(tileindex_df)){
    tileindex_i <- tileindex_df[i,1]
    tileindex_df[i,2] <- paste("/projects/my-public-bucket/AOIs/vero_1deg_tiles_52N52S_land/tile_num_",tileindex_i,".geojson",sep="")
    tileindex_df[i,3] <- paste("https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_",tileindex_i,".geojson",sep="")
    tileindex_df[i,4] <- paste("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles/tile_num_",tileindex_i,"_L4A.gpkg",sep="")
    }

write.csv(tileindex_df, paste("/projects/my-public-bucket/GEDI_global_PA_v2/vero_1deg_tileindex/tileindex_",iso3,".csv", sep=""))

In [20]:
tileindex_df <- read.csv(paste("/projects/my-public-bucket/GEDI_global_PA_v2/vero_1deg_tileindex/tileindex_",iso3,".csv", sep=""))
head(tileindex_df)

,X,tileindex,tilenames,s3path,l4path
,<int>,<int>,<chr>,<chr>,<chr>
1,1,53535,/projects/my-public-bucket/AOIs/vero_1deg_tiles_52N52S_land/tile_num_53535.geojson,https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_53535.geojson,https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles/tile_num_53535_L4A.gpkg


In [12]:
## check if all L4A.gpkg files exist for a given country -- list missing tiles for subsetting
sort(world[world$CONTINENT=='South America',]$ADM0_A3)

[1] "ARG" "BOL" "BRA" "CHL" "COL" "ECU" "FLK" "GUY" "PER" "PRY" "SUR" "URY"
[13] "VEN"

In [32]:
iso3 <- "VEN"
tindex_path <- "/projects/my-public-bucket/GEDI_global_PA_v2/vero_1deg_tileindex/"
tileindex_iso3 <- read.csv(paste(tindex_path,"tileindex_",iso3,".csv", sep=""))
#head(tileindex_iso3)

L4exists <- rep(0,nrow(tileindex_iso3))
for(i in 1:nrow(tileindex_iso3)){
    L4exists[i] <- as.numeric(file.exists(tileindex_iso3[i,"l4path"]))
#    gpkg_table(tileindex_iso3[i,"l4path"])
}
L4exists

missing_tiles <- tileindex_iso3[which(L4exists==0),"tileindex"]
missing_tiles

for(j in 1:length(missing_tiles)){

    print(paste("#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_",
                missing_tiles[j],".geojson'",sep=""))
    }
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_19400.geojson'


[1] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 [38] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 [75] 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[112] 1

[1] 20955

[1] "#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_20955.geojson'"


In [14]:
missing_tiles

integer(0)

In [54]:
library("sf")

Linking to GEOS 3.11.2, GDAL 3.7.0, PROJ 9.2.0; sf_use_s2() is TRUE



In [60]:
install.packages("gpkg")
library("gpkg")

Warning message in install.packages("gpkg"):
“installation of package ‘gpkg’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



ERROR: Error in library("gpkg"): there is no package called ‘gpkg’


In [23]:
iso3_list <- world$ADM0_A3

for(i in 1:length(iso3_list)){
    
    iso3 <- iso3_list[i]
    country <- world[world$ADM0_A3 %in% iso3,]
    aoi <- country
    
    print(iso3)
    
#    if(!file.exists(paste("/projects/my-public-bucket/GEDI_global_PA_v2/vero_1deg_tileindex/tileindex_",iso3,".csv", sep=""))){
        
    if (!identical(crs(aoi), crs(G1deg_tiles))){aoi <- project(aoi, crs(G1deg_tiles))}
        intersecting <- intersect(G1deg_tiles, aoi)
          if (nrow(intersecting) == 0) {print('no intersecting G1deg_tiles found')
          } else {
            tileindex <- intersecting$id
          }
        #print(tileindex)
    
    tilenames <- rep("NA",length(tileindex))
    s3path <- rep("NA",length(tileindex))
    l4path <- rep("NA",length(tileindex))

    tileindex_df <- data.frame(tileindex, tilenames, s3path, l4path)
        for(i in 1:nrow(tileindex_df)){
            tileindex_i <- tileindex_df[i,1]
            tileindex_df[i,2] <- paste("/projects/my-public-bucket/AOIs/vero_1deg_tiles_52N52S_land/tile_num_",tileindex_i,".geojson",sep="")
            tileindex_df[i,3] <- paste("https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_",tileindex_i,".geojson",sep="")
            tileindex_df[i,4] <- paste("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles/tile_num_",tileindex_i,"_L4A.gpkg",sep="")
        }
    write.csv(tileindex_df, paste("/projects/my-public-bucket/GEDI_global_PA_v2/vero_1deg_tileindex/tileindex_",iso3,".csv", sep=""))}
#}

[1] "IDN"
[1] "MYS"
[1] "CHL"
[1] "BOL"
[1] "PER"
[1] "ARG"
[1] "ESB"
[1] "CYP"
[1] "IND"
[1] "CHN"
[1] "ISR"
[1] "PSX"
[1] "LBN"
[1] "ETH"
[1] "SDS"
[1] "SOM"
[1] "KEN"
[1] "PAK"
[1] "MWI"
[1] "TZA"
[1] "SYR"
[1] "SOL"
[1] "FRA"
[1] "SUR"
[1] "GUY"
[1] "KOR"
[1] "PRK"
[1] "MAR"
[1] "SAH"
[1] "CRI"
[1] "NIC"
[1] "COG"
[1] "COD"
[1] "BTN"
[1] "UKR"
[1] "BLR"
[1] "NAM"
[1] "ZAF"
[1] "MAF"
[1] "SXM"
[1] "OMN"
[1] "UZB"
[1] "KAZ"
[1] "TJK"
[1] "LTU"


Warning message:
“[intersect] no intersection”


[1] "no intersecting G1deg_tiles found"
[1] "BRA"
[1] "URY"
[1] "MNG"
[1] "RUS"
[1] "CZE"
[1] "DEU"
[1] "EST"


Warning message:
“[intersect] no intersection”


[1] "no intersecting G1deg_tiles found"
[1] "LVA"


...

[1] "no intersecting G1deg_tiles found"
[1] "FRO"


Warning message:
“[intersect] no intersection”


[1] "no intersecting G1deg_tiles found"
[1] "IOA"
[1] "IOT"
[1] "SGP"
[1] "NFK"
[1] "COK"
[1] "TON"
[1] "WLF"
[1] "WSM"
[1] "SLB"
[1] "TUV"
[1] "MDV"
[1] "NRU"
[1] "FSM"
[1] "SGS"


Warning message:
“[intersect] no intersection”


[1] "no intersecting G1deg_tiles found"
[1] "FLK"
[1] "VUT"
[1] "NIU"
[1] "ASM"
[1] "PLW"
[1] "GUM"
[1] "MNP"
[1] "BHR"
[1] "CSI"
[1] "PGA"
[1] "CLP"
[1] "MAC"
[1] "ATC"
[1] "BJN"
[1] "SER"
[1] "SCR"
